In [93]:
# Import dependencies
import pandas as pd
import os

In [94]:
# Below are the ABSOLUTE paths to the csv files (since they are too big for git without large file handling)
    # PATHS WILL BE DIFFERENT FOR EACH COMPUTER

census_file_cd_csd = "C:/Users/kronh/OneDrive/Coding_bootcamp/project_1_datasources/2021_census_data_cd_cds/98-401-X2021005_English_CSV_data.csv"

employment_nums = "C:/Users/kronh/OneDrive/Coding_bootcamp/project_1_datasources/employment_by_occupation/14100389.csv"

wages = "C:/Users/kronh/OneDrive/Coding_bootcamp/project_1_datasources/wage_data_2021_by_region_and_job.csv"

### Cleaning of census data

In [111]:
# Reads census data by division and subdivision into a DataFrame
census_cd_csd_df = pd.read_csv(census_file_cd_csd, encoding='ISO-8859-1')

# Grabs data from Ontario only - regions with Alt Geo Codes starting with 35
census_cd_csd_df = census_cd_csd_df[census_cd_csd_df["ALT_GEO_CODE"].astype(str).str[:2] == '35']

In [112]:
### Creates a list of dictionaries with division codes tied to division names

# Makes a list for dictionaries of division codes and names in Ontario
divisions = []
# Loops through all the rows in the dataframe at the census division level
for index, row in census_cd_csd_df[census_cd_csd_df['GEO_LEVEL'] == 'Census division'].iterrows():
    # Gets the code and name for the census division
    code = row['ALT_GEO_CODE']
    name = row['GEO_NAME']
    # Creates a dictionary to hold the code and name
    dic = {'code': code, 'name': name}
    # Adds the dictionary to the list divisions if it's not already there
    if dic not in divisions:
        divisions.append(dic)

In [125]:
# Grabs the subset of data at the census subdivision level 
census_csd_df = census_cd_csd_df.loc[census_cd_csd_df['GEO_LEVEL'] == 'Census subdivision'].copy().reset_index(drop=True)

# Keeps a subset of columns
census_csd_df = census_csd_df[['ALT_GEO_CODE', 'GEO_NAME', 'CHARACTERISTIC_ID', 'CHARACTERISTIC_NAME', 'C1_COUNT_TOTAL']]

In [126]:
# Adds a column for census division code to the subdivision data, populates it, and resets the index
census_csd_df['Census division code'] = census_csd_df["ALT_GEO_CODE"].astype(str).str[:4]

# Renames the Geo name column
census_csd_df = census_csd_df.rename(columns={'GEO_NAME': "Census subdivision name", "ALT_GEO_CODE":"Census subdivision code"})

In [127]:
### Filter out unnecessary characteristics and put desired ones into columns

# Makes an empty list to hold dictionaries of values
characteristics_by_csd = []

# Series of each unique census subdivision code
subdivisions = census_csd_df['Census subdivision code'].unique()

# List of dictionaries with characteristic IDs and names
ref_list = [
    {'id': 6, 'name': 'Population density (people/square km)'},
    {'id': 1434, 'name': 'Total households by #/room'},
    {'id': 1435, 'name': '1 or fewer per room'},
    {'id': 1436, 'name': '>1 per room'},
    {'id': 1437, 'name': 'Total households by housing suitability'},
    {'id': 1438, 'name': 'Suitable housing'},
    {'id': 1439, 'name': 'Unsuitable housing'},
    {'id': 1465, 'name': 'Total owned and rented households >0 income'},
    {'id': 1466, 'name': '<30 percent of income spent on shelter'},
    {'id': 1467, 'name': '>=30 percent of income spent on shelter'},
    {'id': 1468, 'name': '30 - <100 percent of income spent on shelter'},
    {'id': 2611, 'name': 'Total commuting duration'},
    {'id': 2612, 'name': 'Commute <15 min'},
    {'id': 2613, 'name': 'Commute 15-29 min'},
    {'id': 2614, 'name': 'Commute 30-44 min'},
    {'id': 2615, 'name': 'Commute 45-59 min'},
    {'id': 2616, 'name': 'Commute >=60 min'}
]

# Iterates through the list of subdivisions
for subdivision in subdivisions:
    # Sets up a dictionary to go into the list and hold the characteristics of this subdivisioon
    characteristics = {'Census subdivision code': subdivision}
    # Grabs all the data pertaining to the given subdivision
    subdivision_data = census_csd_df.loc[census_csd_df['Census subdivision code'] == subdivision]
    # Iterates through the rows of the subdivision data
    for index, row in subdivision_data.iterrows():
        # Checks to see if the characteristic id is equal to one in the reference list 
        if int(row['CHARACTERISTIC_ID']) in list(map(lambda x: x["id"], ref_list)):
            # Finds the index of the dictionary with the found ID
            position = list(map(lambda x: x["id"], ref_list)).index(int(row['CHARACTERISTIC_ID']))
            # Assigns characteristic with ID to the characteristics dictionary
            characteristics[ref_list[position]['name']] = row['C1_COUNT_TOTAL']
    # Adds the dictionary to the characteristics list
    characteristics_by_csd.append(characteristics)

# Makes a DataFrame from the characteristics list
characteristics_df = pd.DataFrame(characteristics_by_csd)

In [128]:
# Drops the CHARACTERISTIC_ID, CHARACTERISTIC_NAME, and C1_COUNT_TOTAL from the census dataframe since that's captured in the characteriztics dataframe now 
census_csd_df = census_csd_df[['Census subdivision code', 'Census subdivision name', 'Census division code']]

# Drops all duplicate rows (and there are a lot now that characteristics have been removed)
census_csd_df = census_csd_df.drop_duplicates(keep='first')

# Merges the census dataframe with the characteristics dataframe so all interesting data is in columns
expanded_census = census_csd_df.merge(characteristics_df, on='Census subdivision code')

In [129]:
### Adds a column for census division name to the subdivision data and populates it

# Adds an empty column for census division name
expanded_census['Census division name'] = ''

# Populates the census division name based on the census division code column 
for index, row in expanded_census.iterrows():
    #(Todo: rewrite with find + lambda, time permitting)
    # Iterates through the dictionaries in the divisions list
    for item in divisions:
        # Checks to see which division code appears in the row 
        if int(row['Census division code']) == int(item['code']):
            # Sets the division name equal to the name in the dictionary with the proper division code
            expanded_census.at[index,'Census division name'] = item['name']

In [130]:

### Adds columns for economic region name and code to the subdivision data and populates them

# Points to a csv with the census divisions in each economic region
er_breakdown = 'Resources/economic_regions_breakdown.csv'

# Puts the economic region breakdown into a DataFrame
er_cd_df = pd.read_csv(er_breakdown)

# Makes empty columns for economic region data in the census dataframe
expanded_census[['Economic region code', 'Economic region name']] = ["", ""]

## Populates economic region columns in the census dataframe
# Iterates over the rows in the census dataframe
for index, row in expanded_census.iterrows():
    # Iterates over the rows in the economic region breakdown dataframe
    for index_2, row_2 in er_cd_df.iterrows():
        # Checks to see if the census division codes on the two rows are equal 
        if str(row['Census division code']).strip() == str(row_2['CD']).strip():
            # Sets economic region code equal to the code in the ER breakdown dataframe
            expanded_census.at[index,'Economic region code'] = row_2['ERC']
            # Sets economic region name equal to the name in the ER breakdown dataframe
            expanded_census.at[index,'Economic region name'] = row_2['ER']

# Reorders the columns
expanded_census = expanded_census[['Economic region code', 'Economic region name', 'Census division code', 'Census division name', 'Census subdivision code', 'Census subdivision name',
        'Population density (people/square km)', 'Total households by #/room', '1 or fewer per room', '>1 per room', 'Total households by housing suitability', 'Suitable housing',
       'Unsuitable housing', 'Total owned and rented households >0 income', '<30 percent of income spent on shelter', '>=30 percent of income spent on shelter',
       '30 - <100 percent of income spent on shelter', 'Total commuting duration', 'Commute <15 min', 'Commute 15-29 min', 'Commute 30-44 min', 'Commute 45-59 min', 'Commute >=60 min']]

# Shows the DataFrame
expanded_census.head()

,Economic region code,Economic region name,Census division code,Census division name,Census subdivision code,Census subdivision name,Population density (people/square km),Total households by #/room,1 or fewer per room,>1 per room,...,Total owned and rented households >0 income,<30 percent of income spent on shelter,>=30 percent of income spent on shelter,30 - <100 percent of income spent on shelter,Total commuting duration,Commute <15 min,Commute 15-29 min,Commute 30-44 min,Commute 45-59 min,Commute >=60 min
0,3510,Ottawa,3501,"Stormont, Dundas and Glengarry, United countie...",3501005,"South Glengarry, Township (TP)",22.0,5430.0,5390.0,35.0,...,5160.0,4500.0,660.0,605.0,4945.0,1485.0,2030.0,825.0,250.0,360.0
1,3510,Ottawa,3501,"Stormont, Dundas and Glengarry, United countie...",3501007,"Akwesasne (Part) 59, Indian reserve (IRI)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3510,Ottawa,3501,"Stormont, Dundas and Glengarry, United countie...",3501011,"South Stormont, Township (TP)",30.3,5415.0,5395.0,20.0,...,5295.0,4725.0,570.0,505.0,5510.0,1355.0,2660.0,775.0,275.0,435.0
3,3510,Ottawa,3501,"Stormont, Dundas and Glengarry, United countie...",3501012,"Cornwall, City (CY)",777.9,21390.0,21095.0,290.0,...,21315.0,16605.0,4705.0,4455.0,15700.0,9380.0,4145.0,910.0,345.0,925.0
4,3510,Ottawa,3501,"Stormont, Dundas and Glengarry, United countie...",3501020,"South Dundas, Municipality (MU)",21.2,4650.0,4600.0,50.0,...,4490.0,3860.0,625.0,580.0,3895.0,1170.0,970.0,760.0,465.0,530.0


### Cleaning of wages by job and economic region

In [131]:
# Reads the wages csv into a DataFrame
wages_df = pd.read_csv(wages)

# Filters wages DataFrame for only Ontario data 
wages_df = wages_df.loc[wages_df['PROV'] == "ON"]
# Removes data for Ontaio as a whole 
wages_df = wages_df.loc[wages_df['ER_Code_Code_RE'] != 'ER35']

# Keeps a subset of the columns and resets the index
wages_df = wages_df[['NOC_Title', "ER_Code_Code_RE", "ER_Name_Nom_RE", "Low_Wage_Salaire_Minium", "Median_Wage_Salaire_Median", "High_Wage_Salaire_Maximal"]].reset_index(drop=True)

# Renames columns 
wages_df = wages_df.rename(columns={'NOC_Title':'National Occupational Classification (NOC)',"ER_Code_Code_RE":"Economic region code", "ER_Name_Nom_RE": "Economic region name", "Low_Wage_Salaire_Minium": "Minimum wage", "Median_Wage_Salaire_Median":"Median wage", "High_Wage_Salaire_Maximal": "Maximum wage"})

# Removes 'ER' from Economic Region code values
wages_df["Economic region code"] = wages_df["Economic region code"].str[2:]

wages_df.head()

,National Occupational Classification (NOC),Economic region code,Economic region name,Minimum wage,Median wage,Maximum wage
0,Legislators,3510,Ottawa,NaN,NaN,NaN
1,Legislators,3515,Kingston--Pembroke,NaN,NaN,NaN
2,Legislators,3520,Muskoka--Kawarthas,NaN,NaN,NaN
3,Legislators,3530,Toronto,NaN,NaN,NaN
4,Legislators,3540,Kitchener--Waterloo--Barrie,NaN,NaN,NaN


### Cleaning of number employed by job and economic region

In [132]:
# Reads employment data into a dataframe
employment_df = pd.read_csv(employment_nums)

# Removes data for Canada as a whole
employment_df = employment_df.loc[employment_df['GEO'] != 'Canada']

# Splits the Geo line into Economic Region and Province
#   Province-level data doesn't have a split, so the povince is assigned to the economic region and province is empty. This would be a problem, except that I only WANT data at the economic region level
employment_df[['Economic Region', 'Province']] = employment_df["GEO"].str.split(", ", expand = True)

# Keeps only data for Ontario at an economic region level 
employment_df = employment_df.loc[employment_df['Province'] == 'Ontario']

# Adds an empty column for Economic Region Code
employment_df['Economic region code'] = ''

### Populates the Economic region code column

# Makes unique Economic region names and codes into a separate dataframe; removes duplicates and resets the index
er_df = er_cd_df.copy()[['ERC', 'ER']].drop_duplicates(keep='first').reset_index(drop=True)

# Iterates over rows in employment dataframe column
for index, row in employment_df.iterrows():
    # Iterates through the rows in the economic region codes dataframe
    for index_2, row_2 in er_df.iterrows():
        # Checks to see which division code appears in the row of the empployment dataframe
        if str(row['Economic Region']).strip() == str(row_2['ER']).strip():
            # Sets the division name equal to the name in the dictionary with the proper division code
            employment_df.at[index,'Economic region code'] = row_2['ERC']
    
# Splits the NOC column into name and number
employment_df[['National Occupational Classification (NOC)',"NOC code"]] = employment_df['National Occupational Classification (NOC)'].str.split("[", expand=True)

# Removes space at the end of NOC column
employment_df['National Occupational Classification (NOC)'] = employment_df['National Occupational Classification (NOC)'].str.strip()

# Removes end bracket from NOC code column
employment_df["NOC code"] = employment_df["NOC code"].str[:-1]

# Filters columns to keep
employment_df = employment_df[['National Occupational Classification (NOC)', "NOC code", 'Economic region code', 'Economic Region', 'VALUE']]

# Renames some columns
employment_df = employment_df.rename(columns={'Economic Region': "Economic region name", "VALUE":"Number Employed (thousands)"})

# Resets the index
employment_df = employment_df.reset_index(drop=True)

# Shows the head of the dataframe
employment_df.head()

,National Occupational Classification (NOC),NOC code,Economic region code,Economic region name,Number Employed (thousands)
0,"Total employed, all occupations",None,3510,Ottawa,640.5
1,Management occupations,0,3510,Ottawa,74.0
2,Senior management occupations,00,3510,Ottawa,6.0
3,Specialized middle management occupations,01-05,3510,Ottawa,30.2
4,Middle management occupations in retail and wh...,06,3510,Ottawa,22.2


### Merge of employment and wage DataFrames